In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [6]:
def read_csv(file_path):
    df = pd.read_csv(file_path)
    X = np.array(df["sentence"])
    Y = np.array(df["label"], dtype=int)
    return X,Y

In [7]:
X_train , Y_train = read_csv("/content/dataset/train.csv")
X_test , Y_test = read_csv("/content/dataset/test.csv")

In [8]:
index = 7
X_train[index], Y_train[index]

('congratulations on your acceptance', np.int64(2))

In [9]:
def label_to_emoji(label):
    emojies = ["💗","⚽","😀","🥺","🍴"]
    return emojies[label]

In [10]:
index = 7
X_train[index], label_to_emoji(Y_train[index])

('congratulations on your acceptance', '😀')

In [11]:
Y_train

array([3, 2, 3, 0, 4, 0, 3, 2, 3, 1, 3, 3, 1, 3, 2, 3, 2, 3, 1, 2, 3, 0,
       2, 2, 2, 1, 4, 2, 2, 4, 0, 3, 4, 2, 0, 3, 2, 2, 3, 4, 2, 2, 0, 2,
       3, 0, 3, 2, 4, 3, 0, 3, 3, 3, 4, 2, 1, 1, 1, 2, 3, 1, 0, 0, 0, 3,
       4, 4, 2, 2, 1, 2, 0, 3, 2, 2, 0, 0, 3, 1, 2, 1, 2, 2, 4, 3, 3, 2,
       4, 0, 0, 0, 3, 3, 3, 2, 0, 1, 2, 3, 0, 2, 2, 2, 3, 2, 2, 2, 4, 1,
       1, 3, 3, 4, 1, 2, 1, 1, 3, 1, 0, 4, 0, 3, 3, 4, 4, 1, 4, 3, 0, 2])

In [15]:
unique, counts = np.unique(Y_train, return_counts=True)
print(unique)
print(counts)

[0 1 2 3 4]
[22 19 38 36 17]


In [19]:
max_len = len(max(X_train, key=len).split(" "))
max_len

10

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [21]:
!unzip -q glove.6B -d glov.6B

In [32]:
f = open("/content/glov.6B/glove.6B.50d.txt", encoding="utf-8")

word_vectors = {}
for line in f:
  line = line.strip().split()
  word = line[0]
  vector = np.array(line[1:], dtype=np.float64)
  word_vectors[word] = vector


In [35]:
word_vectors["snake"]

array([ 0.49251 , -0.24279 , -0.49748 ,  0.28443 ,  0.16984 ,  0.61016 ,
        0.20294 , -0.19734 ,  0.93474 , -0.11809 , -0.26342 ,  0.97142 ,
        1.0427  ,  0.60017 , -0.46936 ,  0.10087 ,  0.60649 ,  1.1277  ,
       -1.1823  , -0.29334 , -0.72885 , -0.46904 ,  1.1104  ,  0.27504 ,
        0.48043 , -1.3031  , -0.58713 ,  0.90264 ,  0.089552, -0.60348 ,
        1.1117  , -0.85367 , -0.13902 ,  0.87767 , -0.19307 ,  0.10299 ,
       -0.83688 , -0.87202 ,  0.46529 , -0.22325 , -0.49207 ,  0.33727 ,
       -0.49699 ,  0.95006 ,  0.75007 , -0.21252 ,  0.47244 , -1.4552  ,
        0.11704 , -0.61483 ])

In [45]:
def sentence_to_avg(sentence):
    try:
      sentence = sentence.lower()
      words = sentence.strip().split(" ")

      sum_vectors = np.zeros((50, ))
      for word in words:
          sum_vectors += word_vectors[word]

      avg_vector = sum_vectors / len(words)
      return avg_vector
    except:
      print("I can't")
      return None

In [37]:
sentence_to_avg("l love programming too much")

array([ 0.090914  ,  0.3313742 ,  0.0398186 ,  0.073718  ,  0.202272  ,
        0.4290412 , -0.250612  , -0.66489992, -0.501244  ,  0.58114804,
       -0.0855058 ,  0.444856  , -0.294552  , -0.0973736 ,  0.3708738 ,
        0.1069    , -0.010002  ,  0.4427686 , -0.0891482 , -0.5012138 ,
       -0.108782  ,  0.303728  ,  0.3217166 ,  0.35265   ,  0.493324  ,
       -0.964564  , -0.4649016 ,  0.147496  ,  0.594362  , -0.181565  ,
        3.07144   ,  0.417246  ,  0.085779  , -0.108112  , -0.1361006 ,
       -0.125394  , -0.17982   , -0.124098  , -0.125486  , -0.25191   ,
        0.2076562 ,  0.147794  , -0.311968  ,  0.286256  , -0.2523147 ,
        0.2400116 ,  0.1322328 , -0.04640632, -0.077026  ,  0.67215   ])

In [46]:
X_train_avg = []
for x_train in X_train:
    X_train_avg.append(sentence_to_avg(x_train))

X_train_avg = np.array(X_train_avg)

X_train_avg.shape

(132, 50)

In [47]:
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes=5)

In [48]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(5, input_shape=(50,), activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [52]:
model.fit(X_train_avg, Y_train_one_hot, epochs=200)

Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.2696 - loss: 1.7058
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2310 - loss: 1.6737 
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2464 - loss: 1.6783 
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2741 - loss: 1.6286 
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3039 - loss: 1.5790 
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2705 - loss: 1.5806 
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2602 - loss: 1.5732 
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2717 - loss: 1.5268  
Epoch 9/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2937 - loss: 1.5090 
Epoch 10/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2888 - loss: 1.5086 
Epoch 11/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3303 - loss: 1.5155 
Epoch 12/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3422 - l

In [56]:
my_test = "i love you my friends"

my_test_avg = sentence_to_avg(my_test)
my_test_avg = np.array([my_test_avg])
result = model.predict(my_test_avg)

y_pred = np.argmax(result)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
0


In [57]:
label_to_emoji(y_pred)

'💗'